In [ ]:
#.\.venv\Scripts\Activate.ps1

import pandas as pd
import numpy as np
from pathlib import Path

bases_path = Path("bases")

checklist = bases_path / "HERING_FRANQUIA_checklist_RA_SMOOTH_0_2_2025-12-03.xlsx"
cancelado = bases_path / "cancelado.xlsx"
relatorio = bases_path / "relatorio.xlsx"
df_check = pd.read_excel(checklist, engine="calamine")
df_cancelado = pd.read_excel(cancelado, engine="calamine")
df_relatorio = pd.read_excel(relatorio, header=5, engine="calamine")




In [ ]:
df_check['DATA']

In [ ]:
# Itera sobre o nome de cada coluna no seu DataFrame
for coluna in df_check.columns:
    # Verifica se a coluna é do tipo 'object', que é como o pandas armazena strings
    if df_check[coluna].dtype == 'object':
        print(f"Limpando espaços da coluna: {coluna}")
        # Se for uma coluna de texto, remove os espaços em branco do início e do fim
        df_check[coluna] = df_check[coluna].str.strip()

print("\nColunas de texto foram limpas.")

In [ ]:
df_check.columns

In [ ]:
df_output = (df_check.groupby(['FILIAL'])[['VELOCIDADE_VENDA']].sum().query('VELOCIDADE_VENDA <= 0').reset_index())
df_output.head()

In [ ]:
df_output_dois = (df_check.groupby(['FILIAL'])[['LOJA']].sum().query('LOJA < 0').reset_index())
df_output_dois.head()

In [ ]:
df_cancelado.columns

In [ ]:
df_relatorio.columns

In [ ]:
df_relatorio.head()

In [ ]:
df_relatorio.columns

In [ ]:
df_relatorio["Atendido (SAP)"].sum()

In [ ]:
df_relatorio["key"] = df_relatorio["Cod. Loja"].astype(str) + "_" + df_relatorio["Artigo"].astype(str) + "_" + df_relatorio["Cor"].astype(str) + "_" + df_relatorio["Tamanho"].astype(str)

In [ ]:
colunas_relatorio = ['key','Atendido (SAP)']

convert_dict = {'key': str,
                'Atendido (SAP)': np.int64}
df_relatorio = df_relatorio[colunas_relatorio]

df_relatorio = df_relatorio.astype(convert_dict)

df_relatorio.head()

In [ ]:
df_relatorio.info()

In [ ]:
df_relatorio = df_relatorio.groupby('key')['Atendido (SAP)'].sum().reset_index()

In [ ]:
df_relatorio

In [ ]:
df_relatorio['Atendido (SAP)'].sum()

In [ ]:
df_check["key"] = df_check["FILIAL"].astype(str) + "_" + df_check["PRODUTO"].astype(str) + "_" + df_check["COR"].astype(str) + "_" + df_check["TAM"].astype(str)

In [ ]:
filtro_região = df_check['REGIAO'].isin(['NORTE', 'NORDESTE'])

filtro_ressuprir = df_check['RESSUPRIR'] > 0 

df_check_filtrado = df_check[filtro_região & filtro_ressuprir]

df_resultado = df_check_filtrado.groupby(['FILIAL','key'])[['EMBALADO', 'RESERVADO', 'TRANSITO']].sum().reset_index()

In [ ]:
df_cancelado.columns

In [ ]:
print("--- ANTES DA LIMPEZA ---")
df_cancelado.info()

# 1. Limpa os nomes das colunas, removendo espaços extras
df_cancelado.columns = df_cancelado.columns.str.strip()

# 2. Converte 'DtCancel' para o tipo data
#    'errors='coerce'' trata erros, transformando-os em NaT (Not a Time)
df_cancelado['DtCancel'] = pd.to_datetime(df_cancelado['DtCancel'], errors='coerce')

# 3. (Opcional mas recomendado) Converte a coluna 'Pecas' para o tipo 'Int64'
#    Isso a mantém como número inteiro, mesmo que existam valores nulos (NaN).
df_cancelado['Pecas'] = df_cancelado['Pecas'].astype('Int64')


print("\n\n--- DEPOIS DA LIMPEZA ---")
# O método .info() é excelente para verificar os nomes das colunas e seus tipos
df_cancelado.info()

In [ ]:
df_cancelado.columns

In [ ]:
df_cancelado["key"] = df_cancelado["PontoVda"].astype(str) + "_" + df_cancelado["Artigo"].astype(str) + "_" + df_cancelado["Tamanho"].astype(str).astype(str)

df_cancelado = df_cancelado.groupby('key')['Pecas'].sum().reset_index()

In [ ]:
resultado_final = df_resultado.merge(df_relatorio, how='left', on='key')
resultado_final['Atendido (SAP)'] = resultado_final['Atendido (SAP)'].fillna(0).astype(np.int64)

resultado_final

In [ ]:
resultado_final = resultado_final.merge(df_cancelado, how='left', on='key')
resultado_final['Pecas'] = resultado_final['Pecas'].fillna(0).astype(np.int64)
resultado_final

In [ ]:
resultado_final["verificação"] = np.where(
    (resultado_final['EMBALADO'] + 
     resultado_final['RESERVADO'] +
       resultado_final['TRANSITO']
    )
    >= resultado_final['Atendido (SAP)'], 'OK', 'VERIFICAR'
)

In [ ]:
resultado_final

In [ ]:
resultado_final_filtrado = resultado_final[resultado_final['verificação'].isin(['VERIFICAR'])]

resultado_final_filtrado

In [ ]:
colunas_para_somar = ['EMBALADO', 'RESERVADO', 'TRANSITO', 'Atendido (SAP)', 'Pecas']

# Realizando o Group By e a soma
df_agrupado = resultado_final_filtrado.groupby('FILIAL')[colunas_para_somar].sum().reset_index()

# Renomeando a coluna 'Pecas' para 'Pecas Cancelado' conforme você pediu
df_agrupado = df_agrupado.rename(columns={'Pecas': 'Pecas Cancelado'})

# Visualizar o resultado
df_agrupado